In [15]:
import bs4
import lxml
import pandas as pd
import urllib
from urllib import request
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil

In [2]:
#Le 38 Riv
url = "https://www.38riv.com/billetterie-2/"
request_text = request.urlopen(url).read()
page = bs4.BeautifulSoup(request_text, "lxml")
link_list = page.find_all('a', {"class": "open_tickets"})
print(link_list)

NameError: name 'request' is not defined

In [53]:
#L'international
url_2 = "https://www.linternational.fr/agenda"
request_text = request.urlopen(url_2).read()
page = bs4.BeautifulSoup(request_text, "lxml")
exhibitions = page.find_all('div', class_ = "Zz5cHg")
information = ['nom', 'date','heure','prix','etablissement','adresse','arrondissement']
D = {key:[] for key in information}
for exhibition in exhibitions:
    info = exhibition.img.get('alt').split(' - ')
    D['date'].append(info[0])
    D['heure'].append(info[1])
    D['prix'].append(info[2])
n = len(D['date'])

p = []
for prix in D['prix']:
    p.append(prix[:-1])
D['prix'] = p

D['etablissement'] = ["L'international"]*n
D['nom'] = ['unknown']*n
D['adresse'] = ["5 rue Moret"]*n
D['arrondissement'] = [11]*n

data = pd.DataFrame(D)
data.tail()
#international_csv = data.to_csv('C:\\Users\\LENOVO\\Desktop\\data_international.csv', index =False, encoding = 'utf-8')



,nom,date,heure,prix,etablissement,adresse,arrondissement
25,unknown,Mardi 12 décembre,20h,8,L'international,5 rue Moret,11
26,unknown,Mercredi 13 décembre,20h,7,L'international,5 rue Moret,11
27,unknown,Jeudi 14 décembre,20h,8,L'international,5 rue Moret,11
28,unknown,Jeudi 14 décembre,00h,5,L'international,5 rue Moret,11
29,unknown,Vendredi 15 décembre,00h,7,L'international,5 rue Moret,11


In [16]:
#Tentative de recup des covers
url_3 = "https://www.linternational.fr/agenda"
request_text = request.urlopen(url_3).read()
page = bs4.BeautifulSoup(request_text, "lxml")
exhibitions = page.find_all('div', class_ = "Zz5cHg")
L =[]
for exhibition in exhibitions:
    L.append(exhibition.img.get('src'))
print(L)

r = request(L[0], stream = True)
if r.status_code == 200:                     #200 status code = OK
   with open("images/book1.jpg", 'wb') as f: 
      r.raw.decode_content = True
      shutil.copyfileobj(r.raw, f)

img = mpimg.imread('images/book1.jpg')
imgplot = plt.imshow(img)
plt.show()

['https://static.wixstatic.com/media/3f3a7f_d6b32b0656354f06b9132f271c24431c~mv2.jpeg/v1/fill/w_147,h_77,al_c,q_80,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_d6b32b0656354f06b9132f271c24431c~mv2.jpeg', 'https://static.wixstatic.com/media/3f3a7f_044cb6ea035f403dbbcdf47b07f8a81a~mv2.png/v1/fill/w_49,h_49,al_c,q_85,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_044cb6ea035f403dbbcdf47b07f8a81a~mv2.png', 'https://static.wixstatic.com/media/3f3a7f_b423d6bd848d42c7be90559302b65e5b~mv2.png/v1/fill/w_49,h_26,al_c,q_85,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_b423d6bd848d42c7be90559302b65e5b~mv2.png', 'https://static.wixstatic.com/media/3f3a7f_84d7823d333c4fcc9194fb4445dbfae0~mv2.jpeg/v1/fill/w_147,h_77,al_c,q_80,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_84d7823d333c4fcc9194fb4445dbfae0~mv2.jpeg', 'https://static.wixstatic.com/media/3f3a7f_4f631e99e059424e97566abf6db2b217~mv2.png/v1/fill/w_49,h_26,al_c,q_85,usm_0.66_1.00_0.01,blur_2,enc_auto/3f3a7f_4f631e99e059424e97566abf6db2b217~mv2.png', 'https:/

TypeError: 'module' object is not callable